- 데이터 수집 경로: https://www.kaggle.com/datasets/sriramr/fruits-fresh-and-rotten-for-classification
- 이미지는 train 폴더와 test 폴더로 구분되고, 사과, 바나나, 오렌지 과일을 신선, 썩은 과일로 분류되어 있다.
- 이 중 사과 폴더를 이용해서 모델에 사용할 데이터셋을 만든다.
- 이미지 인식에 좋은 성능을 보이는 신경망 모델로 합성곱 신경망을 사용한다.


## 절차 ##


- 합성곱 신경망으로 이미지의 특징을 추출한다.
- 사전 훈련된 모델인 VGG16 모델이나 Inception v3 모델을 이용해 특징을 추출한다.
- 추출된 특징값을 완전연결층의 입력으로 받아 분류 모델을 설계한다.
- 신선한 사과(freshapples)와 썩은 사과(rottenapples)인 2개로 분류하므로 출력층 2개의 노드값 중 큰 값으로 예측한다.
- 예를 들어 출력층 결과가 신선한 사과로 예측하는 확률값이 0.3, 썩은 사과로 예측한 확률값이 0.7이라면 큰 값인 썩은 사과로 예측한다.
- https://mengu.tistory.com/23


## 합성곱 신경망 ##


- 합성곱 신경망(Convolution Neural Network)는 이미지, 비디오 인식에 좋은 성능을 보여주는 딥러닝 기술 중 하나다.
- 구조는 크게 합성곱층, 풀링층, 완전연결층으로 구성된다.
- 합성곱층에서는 이미지 특징을 추출하고, 풀링층에서는 추출된 특징 중에 중요한 부분만 추출해 데이터의 크기를 줄이는 역할을 수행한다.
- 이러한 과정을 반복하면 큰 크기의 이미지 데이터에서 핵심적인 특징만 추출하게 되므로 분류하는데 큰 도움을 준다.
- 완전 연결층에서는 이미지의 분류를 수행하는 역할을 한다.
- 합성곱층과 풀링층으로 추출된 특징값들을 입력으로 받이 클래스 분류를 수행한다.
- 이때 이미지 특징값은 완전연결층의 입력층으로 들어와 학습시켜 출력층에서 확률값 중 가장 큰 확률값을 분류로 사용한다.


## 사전 훈련된 모델 ##


- 사전 훈련된 모델은 대규모 이미지 분류 문제를 위해 대량의 데이터셋으로 미리 훈련되어 저장된 신경망이다.
- 사전 훈련된 합성곱 신경망의 훈련에 사용한 원본 데이터셋이 충분히 크다면, 일반적인 이미지 분류 모델로 사용될 수 있다.
- 사전 훈련된 합성곱 신경망들은 이미지 인식대회(ILSVRC)의 대형 이미지 데이터베이스에서 대량의 원본 데이터셋을 이용해 훈련되어 우승한 모델들을 많이 사용한다.


## 전이 학습 ##


- 전이학습은 어떤 목적을 이루기 위해 학습된 모델을 다른 작업에 이용하는 것을 말한다.
- 일반적으로 합성곱 신경망으로 딥러닝 모델을 만들려면 많은 양의 데이터가 필요하지만 현실적으로 충분히 큰 데이터셋을 얻는 것은 쉽지 않다.
- 사전 훈련된 신경망 모델의 전체 또는 일부를 가져와 해결하려고 하는 과제에 맞게 최적화해서 사용할 수 있다.
- 예를 들어 썩은 사과를 분류하는 경우, 사전 훈련된 이미지 분류 모델을 사용하여 이미지의 특징을 추출할 수 있다.
- 사전 훈련된 분류 모델은 일반적으로 ImageNet 데이터셋에서 훈련된다.
- 여기서는 다양한 종류의 이미지로 구성돼 있기 때문에, 사전 훈련된 이미지 분류 모델은 다양한 이미지의 특징을 추출할 수 있다.


In [24]:
# 드라이브 접근 
# 작업 드라이브 이동 
from PIL import Image
import os

folder_path = './fruit_data'
image_extensions = {'png'}

## ImageDataGenerator ##


- 이미지를 읽고, 전처리하고, 증강하여 모델학습에 사용


- 사용 파라미터
 - rescale: 이미지 픽셀값을 정규화함. 일반적으로 0 ~ 1사이 정규화 값은 1./255
 - rotate_range: 이미지를 지정된 각도 범위 내에서 무작위로 회전. 20으로 설정하면 -20 ~ 20도 사이로 회전
 - width_shift_range, height_shift_range: 이미지를 가로 및 세로로 이동시킴. 값은 0 ~ 1사이의 비율
 - zoom_range: 이미지를 확대 또는 축소. 0.2로 설정하면 이미지가 0.8배에서 1.2배까지 무작위로 확대축소
 - horizontal_flip, vertical_flip: 이미지를 수평 또는 수직으로 뒤집음


In [25]:
# 훈련 데이터 처리 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
    directory=os.path.join(folder_path, 'train'), # train 폴더에서 이미지 호출 후 전처리 적용 
    target_size = (64, 64), # 64 * 64
    batch_size = 32, # 한번에 32개 이미지 처리 
    shuffle=True, # 무작위 선택 
    class_mode='categorical' #여러 이미지를 분류하기 떄문에 다중 클래스 모드 사용 
)

Found 400 images belonging to 2 classes.


In [26]:


test_datagen = ImageDataGenerator(rescale=1.0 / 255)

test_set = test_datagen.flow_from_directory(
    directory=os.path.join(folder_path, 'test'),
    target_size=(64, 64),  # 64 * 64
    batch_size=32,  # 한번에 32개 이미지 처리
    shuffle=False,  # 무작위 선택
    class_mode="categorical",  # 여러 이미지를 분류하기 떄문에 다중 클래스 모드 사용
)


Found 40 images belonging to 2 classes.


In [27]:
# 합성곱 신경망 모델 생성 
# 사전 훈련된 합성곱 신경망 모델인 VGG16을 불러 이미지 특징을 추출 
# include_top= False: 완전 연결층을 제외한 합성곱층, 풀링층만 수행해 특징 추출
# weight = 'imagenet': 모델에 imagenet 데이터셋으로 사전 훈련된 가중치 사용 

from keras.applications.vgg16 import VGG16

# vgg 모델 생성 

vgg = VGG16(include_top=False, weights='imagenet', input_shape=(64, 64, 3))
vgg.summary()

# vgg 모델의 모든 층의 가중치가 중첩해서 업데이트 되지 않도록 함 
# 모든 층의 가중치가 업데이트 되지 않도록 해서 모델 학습시 불필요한 계산 줄임 
for layer in vgg.layers:
    layer.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)           │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 64, 64, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 64, 64, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 32, 32, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 16, 16, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 16, 16, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 16, 16, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 8, 8, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 8, 8, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 8, 8, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 8, 8, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 4, 4, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 2, 2, 512)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# vgg16 모델로 추출한 특징 값을 입력층으로 입력 받아 완전 연결층 신경망 모델 설계
#케라스 완전 연결층 구성 클래스 로드 
from keras.models import Sequential
from keras.layers import Dense, Flatten

In [31]:
model = Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))
